# List of things in pandas I usually forgot

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

In [2]:
from numpy.random import default_rng

import numpy as np
import pandas as pd

rng = default_rng(0)

Generator(PCG64) at 0x7F9E74824820

## MultiIndex accessing when level > 0

In [3]:
a = [
    np.array(["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"]),
    np.array(["one", "two", "one", "two", "one", "two", "one", "two"]),
]
# naming multiindex
# https://stackoverflow.com/a/49505103
a1 = pd.MultiIndex.from_product([["A", "B", "C"], ["E", "F"]], names=["idx_1", "idx_2"])
df = pd.DataFrame(rng.integers(10, size=(6, 8)), index=a1, columns=a)

bar     baz     foo     qux    
            one two one two one two one two
idx_1 idx_2                                
A     E       8   6   5   2   3   0   0   0
      F       1   8   6   9   5   6   9   7
B     E       6   5   5   9   2   8   6   0
      F       3   8   5   0   7   7   8   1
C     E       0   8   0   5   0   2   4   4
      F       4   0   0   1   0   6   5   6

### Method 1: `pd.xs`

In [4]:
df.xs("E", level=1)

bar     baz     foo     qux    
      one two one two one two one two
idx_1                                
A       8   6   5   2   3   0   0   0
B       6   5   5   9   2   8   6   0
C       0   8   0   5   0   2   4   4

### Method 2: `pd.loc` and `slice(None)`
The class `slice(None)` indicates that we want all of `idx_1`. The `"E"` indicates we only want `idx_2` with label `"E"`. Both are enclosed in a tuple which indicates it's for loc row indexing. The `:` indicates we want all columns.

In [5]:
df.loc[(slice(None), "E"), :]

bar     baz     foo     qux    
            one two one two one two one two
idx_1 idx_2                                
A     E       8   6   5   2   3   0   0   0
B     E       6   5   5   9   2   8   6   0
C     E       0   8   0   5   0   2   4   4

## Method 3: `pd.loc` and `pd.IndexSlice` (best method)
Pythonic slice `:` can be used with pd.IndexSlice

In [6]:
df.loc[pd.IndexSlice[:, "E"], :]

bar     baz     foo     qux    
            one two one two one two one two
idx_1 idx_2                                
A     E       8   6   5   2   3   0   0   0
B     E       6   5   5   9   2   8   6   0
C     E       0   8   0   5   0   2   4   4